# xgb alpha

## preprocess

In [1]:
import random
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from autogluon.core.metrics import make_scorer
from autogluon.tabular import TabularDataset, TabularPredictor

from sklearn.model_selection import StratifiedKFold

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

os.makedirs('subs', exist_ok=True)
# import warnings
# warnings.filterwarnings(action='ignore')

In [2]:
seed = 21011928
#724, 990313, 21011928
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(seed) # Seed 고정

In [3]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
infos = pd.read_csv('building_info.csv')
infos = infos.replace('-', np.NaN)
infos[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']] = infos[['태양광용량(kW)', 'ESS저장용량(kWh)', 'PCS용량(kW)']].astype(float)
train_df.head()

,num_date_time,건물번호,일시,기온(C),강수량(mm),풍속(m/s),습도(%),일조(hr),일사(MJ/m2),전력소비량(kWh)
0,1_20220601 00,1,20220601 00,18.6,NaN,0.9,42.0,NaN,NaN,1085.28
1,1_20220601 01,1,20220601 01,18.0,NaN,1.1,45.0,NaN,NaN,1047.36
2,1_20220601 02,1,20220601 02,17.7,NaN,1.5,45.0,NaN,NaN,974.88
3,1_20220601 03,1,20220601 03,16.7,NaN,1.4,48.0,NaN,NaN,953.76
4,1_20220601 04,1,20220601 04,18.4,NaN,2.8,43.0,NaN,NaN,986.40


In [4]:
infos['태양광용량(kW)'] = infos['태양광용량(kW)'].fillna(0)
infos['태양광용량(kW)'] = np.where(infos['태양광용량(kW)'] > 0, 1, 0)
infos['태양광용량(kW)'].value_counts()

0    64
1    36
Name: 태양광용량(kW), dtype: int64

In [5]:
infos.isnull().sum()

건물번호             0
건물유형             0
연면적(m2)          0
냉방면적(m2)         0
태양광용량(kW)        0
ESS저장용량(kWh)    95
PCS용량(kW)       95
dtype: int64

In [6]:
train_df = pd.merge(train_df, infos[['건물번호', '건물유형', '태양광용량(kW)']], how = 'outer',on = '건물번호')
test_df = pd.merge(test_df, infos[['건물번호', '건물유형', '태양광용량(kW)']], how = 'outer',on = '건물번호')
# train_df = pd.merge(train_df, infos[['건물번호', '건물유형', '연면적(m2)', '냉방면적(m2)']], how = 'outer',on = '건물번호')
# test_df = pd.merge(test_df, infos[['건물번호', '건물유형', '연면적(m2)', '냉방면적(m2)']], how = 'outer',on = '건물번호')

In [7]:
date = pd.to_datetime(train_df.일시)
train_df['hour'] = date.dt.hour
train_df['day'] = date.dt.weekday
train_df['month'] = date.dt.month
train_df['week'] = date.dt.weekofyear

date = pd.to_datetime(test_df.일시)
test_df['hour'] = date.dt.hour
test_df['day'] = date.dt.weekday
test_df['month'] = date.dt.month
test_df['week'] = date.dt.weekofyear

# train_df[['강수량(mm)', '풍속(m/s)', '습도(%)']] = train_df.groupby(['month', '건물번호'])[['강수량(mm)', '풍속(m/s)', '습도(%)']].transform(lambda x: x.fillna(x.mean()))

for month in train_df['month'].unique():
    train_df.loc[train_df['month'] == month, ['강수량(mm)', '풍속(m/s)', '습도(%)']] = train_df.loc[train_df['month'] == month, ['강수량(mm)', '풍속(m/s)', '습도(%)']].fillna(train_df.loc[train_df['month'] == month, ['강수량(mm)', '풍속(m/s)', '습도(%)']].mean())
# for month in train_df['month'].unique():
#     train_df.loc[train_df['month'] == month, ['풍속(m/s)', '습도(%)']] = train_df.loc[train_df['month'] == month, ['풍속(m/s)', '습도(%)']].fillna(train_df.loc[train_df['month'] == month, ['풍속(m/s)', '습도(%)']].mean())

C:\Users\USER\AppData\Local\Temp\ipykernel_18256\1945127973.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  train_df['week'] = date.dt.weekofyear
C:\Users\USER\AppData\Local\Temp\ipykernel_18256\1945127973.py:11: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  test_df['week'] = date.dt.weekofyear


In [8]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [9]:
train_df.loc[(train_df['건물번호'] == 95) & (train_df['전력소비량(kWh)'] < 1), '전력소비량(kWh)'] = np.NaN
train_df['전력소비량(kWh)'] = train_df['전력소비량(kWh)'].interpolate(limit_direction='both', method='linear').round(3)

In [10]:
# # 1
# train_df.loc[770:777, '전력소비량(kWh)'] = train_df.loc[602:609, '전력소비량(kWh)'].values
# train_df.loc[939:946, '전력소비량(kWh)'] = train_df.loc[602:609, '전력소비량(kWh)'].values

# # 4
# train_df.loc[7438:7439, '전력소비량(kWh)'] = train_df.loc[7416:7417, '전력소비량(kWh)'].values
# train_df.loc[7440, '전력소비량(kWh)'] = np.nan

# # 4
# # train_df.loc[7438, '전력소비량(kWh)'] = train_df.loc[7438-24*7, '전력소비량(kWh)'].values

# # 11
# train_df.loc[21893, '전력소비량(kWh)'] = train_df.loc[21893-24*7, '전력소비량(kWh)']
# train_df.loc[22058, '전력소비량(kWh)'] = train_df.loc[22058-24*7, '전력소비량(kWh)']
# train_df.loc[22065, '전력소비량(kWh)'] = train_df.loc[22065-24*7, '전력소비량(kWh)']
# train_df.loc[21700:21701, '전력소비량(kWh)'] = train_df.loc[21700-24*7:21701-24*7, '전력소비량(kWh)'].values
# train_df.loc[22243, '전력소비량(kWh)'] = train_df.loc[22243-24*7, '전력소비량(kWh)']

# # 17
# train_df.loc[33888:33901, '전력소비량(kWh)'] = train_df.loc[33888-24*7:33901-24*7, '전력소비량(kWh)'].values

# # 26
# train_df.loc[52089:52090, '전력소비량(kWh)'] = train_df.loc[52089-24*7:52090-24*7, '전력소비량(kWh)'].values

# # 28
# train_df.loc[56450, '전력소비량(kWh)'] = train_df.loc[56450-24*7, '전력소비량(kWh)']

# # 31
# train_df.loc[61955, '전력소비량(kWh)'] = train_df.loc[61955-24*7, '전력소비량(kWh)']

# # 34
# train_df.loc[68973, '전력소비량(kWh)'] = train_df.loc[68973-24*7, '전력소비량(kWh)']

# # 56
# train_df.loc[112384, '전력소비량(kWh)'] = train_df.loc[112384-24*7, '전력소비량(kWh)']

# # 61
# train_df.loc[123132, '전력소비량(kWh)'] = train_df.loc[123132-24*7, '전력소비량(kWh)']

# # 69
# train_df.loc[138904, '전력소비량(kWh)'] = train_df.loc[138904-24*7, '전력소비량(kWh)']

# # 72
# train_df.loc[145424:145425, '전력소비량(kWh)'] = np.nan
# train_df.loc[145786, '전력소비량(kWh)'] = np.nan
# train_df.loc[145954, '전력소비량(kWh)'] = np.nan

# # 95
# train_df.loc[193120:193121, '전력소비량(kWh)'] = train_df.loc[193120-24*7:193121-24*7, '전력소비량(kWh)'].values
# train_df.loc[193133, '전력소비량(kWh)'] = train_df.loc[193133-24*7, '전력소비량(kWh)']

# # 75
# train_df.loc[151303:151314, '전력소비량(kWh)'] = train_df.loc[151303-24*7:151314-24*7, '전력소비량(kWh)'].values
# train_df.loc[151423:151455, '전력소비량(kWh)'] = train_df.loc[151423-24*7:151455-24*7, '전력소비량(kWh)'].values
# train_df.loc[152393, '전력소비량(kWh)'] = train_df.loc[152393-24*7, '전력소비량(kWh)']

# # 70
# # train.loc[142200:142229, '전력소비량(kWh)'] = np.nan
# train_df.loc[142004+169:142004+169+168, '전력소비량(kWh)'] = train_df.loc[142004:142004+168, '전력소비량(kWh)'].values
# train_df.loc[142171:142172, '전력소비량(kWh)'] = train_df.loc[142171-24*7:142172-24*7, '전력소비량(kWh)'].values
# train_df.loc[142340:142379, '전력소비량(kWh)'] = train_df.loc[142340-24*7:142379-24*7, '전력소비량(kWh)'].values
# # train.loc[142200:142229, '전력소비량(kWh)'] = train.loc[142200-24*7:142229-24*7, '전력소비량(kWh)'].values

# train_df['전력소비량(kWh)'] = train_df['전력소비량(kWh)'].interpolate(limit_direction='both', method='linear').round(3)

In [11]:
power_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'day', 'month'], aggfunc = np.mean).reset_index()
train_df['month_day_mean'] = train_df.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.day == x['day']) & (power_mean.month == x['month']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['month_day_mean'] = test_df.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.day == x['day']) & (power_mean.month == x['month']) ,'전력소비량(kWh)'].values[0], axis = 1)

In [12]:
power_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.mean).reset_index()
train_df['day_hour_mean'] = train_df.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['day_hour_mean'] = test_df.apply(lambda x : power_mean.loc[(power_mean.건물번호 == x['건물번호']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour', 'day'], aggfunc = np.std).reset_index()
train_df['day_hour_std'] = train_df.apply(lambda x : power_std.loc[(power_std.건물번호 == x['건물번호']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['day_hour_std'] = test_df.apply(lambda x : power_std.loc[(power_std.건물번호 == x['건물번호']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.mean).reset_index()
train_df['hour_mean'] = train_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물번호 == x['건물번호']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['hour_mean'] = test_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물번호 == x['건물번호']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물번호', 'hour'], aggfunc = np.std).reset_index()
train_df['hour_std'] = train_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물번호 == x['건물번호']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['hour_std'] = test_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물번호 == x['건물번호']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)


power_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour', 'day'], aggfunc = np.mean).reset_index()
train_df['type_day_hour_mean'] = train_df.apply(lambda x : power_mean.loc[(power_mean.건물유형 == x['건물유형']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_day_hour_mean'] = test_df.apply(lambda x : power_mean.loc[(power_mean.건물유형 == x['건물유형']) & (power_mean.hour == x['hour']) & (power_mean.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour', 'day'], aggfunc = np.std).reset_index()
train_df['type_day_hour_std'] = train_df.apply(lambda x : power_std.loc[(power_std.건물유형 == x['건물유형']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_day_hour_std'] = test_df.apply(lambda x : power_std.loc[(power_std.건물유형 == x['건물유형']) & (power_std.hour == x['hour']) & (power_std.day == x['day']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_mean = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour'], aggfunc = np.mean).reset_index()
train_df['type_hour_mean'] = train_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물유형 == x['건물유형']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_hour_mean'] = test_df.apply(lambda x : power_hour_mean.loc[(power_hour_mean.건물유형 == x['건물유형']) & (power_hour_mean.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)

power_hour_std = pd.pivot_table(train_df, values = '전력소비량(kWh)', index = ['건물유형', 'hour'], aggfunc = np.std).reset_index()
train_df['type_hour_std'] = train_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물유형 == x['건물유형']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)
test_df['type_hour_std'] = test_df.apply(lambda x : power_hour_std.loc[(power_hour_std.건물유형 == x['건물유형']) & (power_hour_std.hour == x['hour']) ,'전력소비량(kWh)'].values[0], axis = 1)


### 공휴일 변수 추가
train_df['holiday'] = train_df.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)
train_df.loc[('20220601' <= train_df.일시)&(train_df.일시 < '20220602'), 'holiday'] = 1
train_df.loc[('20220606' <= train_df.일시)&(train_df.일시 < '20220607'), 'holiday'] = 1
train_df.loc[('20220815' <= train_df.일시)&(train_df.일시 < '20220816'), 'holiday'] = 1

test_df['holiday'] = test_df.apply(lambda x : 0 if x['day']<5 else 1, axis = 1)

train_df['sin_time'] = np.sin(2*np.pi*train_df.hour/24)
train_df['cos_time'] = np.cos(2*np.pi*train_df.hour/24)
test_df['sin_time'] = np.sin(2*np.pi*test_df.hour/24)
test_df['cos_time'] = np.cos(2*np.pi*test_df.hour/24)

train_df['THI'] = 9/5*train_df['기온(C)'] - 0.55*(1-train_df['습도(%)']/100)*(9/5*train_df['습도(%)']-26)+32
test_df['THI'] = 9/5*test_df['기온(C)'] - 0.55*(1-test_df['습도(%)']/100)*(9/5*test_df['습도(%)']-26)+32

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

train_df['CDH'] = 0
for num in range(1,101,1):
    temp = train_df[train_df['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    train_df.loc[train_df['건물번호'] == num, 'CDH'] = cdh

test_df['CDH'] = 0
for num in range(1,101,1):
    temp = test_df[test_df['건물번호'] == num]
    cdh = CDH(temp['기온(C)'].values)
    test_df.loc[test_df['건물번호'] == num, 'CDH'] = cdh

In [ ]:
train_df['7_shifted_전력소비량'] = train_df['전력소비량(kWh)'].shift(24*7)
train_df = train_df[train_df.일시 >= '20220608'].reset_index(drop=True)

for i in train_df['건물번호'].unique():
    test_df.loc[test_df['건물번호'] == i, '7_shifted_전력소비량'] = train_df.loc[train_df['건물번호'] == i, '전력소비량(kWh)'][-7*24:].values

In [ ]:
# # 화씨온도
# train_df['temperature_F'] = (train_df['기온(C)'] * 9/5) + 32
# test_df['temperature_F'] = (test_df['기온(C)'] * 9/5) + 32

# 둘째, 넷째 주 일요일
# train_df['2_4_sunday'] = 0
# train_df.loc[('20220612' <= train_df.일시)&(train_df.일시 < '20220613'), '2_4_sunday'] = 1
# train_df.loc[('20220626' <= train_df.일시)&(train_df.일시 < '20220617'), '2_4_sunday'] = 1
# train_df.loc[('20220710' <= train_df.일시)&(train_df.일시 < '20220711'), '2_4_sunday'] = 1
# train_df.loc[('20220724' <= train_df.일시)&(train_df.일시 < '20220725'), '2_4_sunday'] = 1
# train_df.loc[('20220814' <= train_df.일시)&(train_df.일시 < '20220815'), '2_4_sunday'] = 1

# test_df['2_4_sunday'] = 0
# test_df.loc[('20220828' <= test_df.일시)&(test_df.일시 < '20220829'), '2_4_sunday'] = 1

In [ ]:
train_x = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)', '건물유형'])
train_x_type = train_df.drop(columns=['num_date_time', '일시', '일조(hr)', '일사(MJ/m2)', '전력소비량(kWh)'])
train_y = train_df['전력소비량(kWh)']

test_x = test_df[train_x.columns]
test_x_type = test_df[train_x_type.columns]
train_x.head()

,건물번호,기온(C),강수량(mm),풍속(m/s),습도(%),태양광용량(kW),hour,day,month,week,...,type_day_hour_mean,type_day_hour_std,type_hour_mean,type_hour_std,holiday,sin_time,cos_time,THI,CDH,7_shifted_전력소비량
0,1,19.0,1.889045,3.2,61.0,0,0,2,6,23,...,1620.812462,886.546805,1616.129012,909.159339,0,0.000000,1.000000,48.2249,-38.9,1085.28
1,1,18.8,1.889045,2.6,61.0,0,1,2,6,23,...,1608.544000,906.029700,1603.843635,921.827876,0,0.258819,0.965926,47.8649,-45.1,1047.36
2,1,18.5,1.889045,2.6,62.0,0,2,2,6,23,...,1577.389538,900.729730,1575.108376,916.043890,0,0.500000,0.866025,47.4096,-51.4,974.88
3,1,18.1,1.889045,2.2,63.0,0,3,2,6,23,...,1561.107077,900.693834,1553.335094,908.635022,0,0.707107,0.707107,46.7941,-58.0,953.76
4,1,17.2,1.889045,3.2,66.0,0,4,2,6,23,...,1546.894154,885.287208,1543.278965,900.520543,0,0.866025,0.500000,45.6064,-64.9,986.40


## modeling

In [ ]:
# custom objective function for forcing model not to underestimate
def weighted_mse(alpha = 1):
    def weighted_mse_fixed(label, pred):
        residual = (label - pred).astype("float")
        grad = np.where(residual>0, -2*alpha*residual, -2*residual)
        hess = np.where(residual>0, 2*alpha, 2.0)
        return grad, hess
    return weighted_mse_fixed

def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 100


In [ ]:
# train_x_type["hour"] = train_x_type["hour"].astype("category")
# test_x_type["hour"] = test_x_type["hour"].astype("category")

In [ ]:
mape_custom = make_scorer(name='SMAPE',
                                 score_func=SMAPE,
                                 optimum=0,
                                 greater_is_better=False)

In [ ]:
alphas = range(15, 16)
alphas

range(15, 16)

In [ ]:
from tqdm import tqdm

valid_df = train_x.copy()
valid_df['전력소비량(kWh)'] = 0
valid_df['pred'] = 0
pred_df = test_x.copy()
pred_df['전력소비량(kWh)'] = 0

labels = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for i in tqdm(train_x['건물번호'].unique()):
    X = train_x[train_x['건물번호'] == i].drop(columns='건물번호')
    y = train_y[train_x['건물번호'] == i]
    valid_df.loc[train_x['건물번호'] == i, '전력소비량(kWh)'] = y
    y = np.log1p(y)
    x_test = test_x[test_x['건물번호'] == i].drop(columns='건물번호')

    cols = X.columns[X.nunique() != 1]
    X = X[cols]
    x_test = x_test[cols]

    tmp_preds = []
    for alpha in alphas:

        model = XGBRegressor(random_state=seed, tree_method="gpu_hist")

        results = []

        for fold_, (train_index, val_index) in enumerate(skf.split(X, X['day'])):

            x_train, x_valid = X.iloc[train_index], X.iloc[val_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[val_index]


            tr = pd.concat([x_train,y_train],axis=1)
            vl = pd.concat([x_valid,y_valid],axis=1)

            train_data = TabularDataset(tr)
            test_data = TabularDataset(x_test)

            predictor = TabularPredictor(label='전력소비량(kWh)',  eval_metric=mape_custom).fit(train_data,  tuning_data=vl, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, excluded_model_types = ['CAT', 'NN_TORCH' ,'XGB' ,'RF', 'FASTAI', 'KNN', 'XT'])
            pred = predictor.predict(test_data, model='LightGBMLarge')
            
            preds = np.expm1(pred)
            val_pred = predictor.predict(x_valid)
            valid_df.loc[x_valid.index, 'pred'] += np.expm1(val_pred)
            pred = predictor.predict(x_test)
            results.append(pred)

        preds = np.mean(results, axis=0)
        preds = np.expm1(preds)
        tmp_preds.append(preds)

    preds = np.mean(tmp_preds, axis=0)
    pred_df.loc[test_x['건물번호'] == i, '전력소비량(kWh)'] = preds

    labels.append([valid_df.loc[train_x['건물번호'] == i, '전력소비량(kWh)'], preds])

# 2.171737450532235
valid_df['pred'] = valid_df['pred'] / len(alphas)
val_score = SMAPE(valid_df['전력소비량(kWh)'], valid_df['pred'])
val_score

  0%|          | 0/100 [00:00<?, ?it/s]No path specified. Models will be saved in: "AutogluonModels\ag-20230827_130643\"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_130643\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   126.08 GB / 498.62 GB (25.3%)
Train Data Rows:    1497
Train Data Columns: 23
Tuning Data Rows:    375
Tuning Data Columns: 23
Label Column: 전력소비량(kWh)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (8.530014644176218, 6.674005681880703, 7.81407, 0.44399)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary', 

Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    7741.67 MB
	Train Data (Original)  Memory Usage: 0.34 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
			Note: Converting 1 features to boolean dtype as they only contain 2 unique values.
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 18 | ['기온(C)', '강수량(mm)', '풍속(m/s)', '습도(%)', 'month_day_mean', ...]
		('int', [])   :  5 | ['hour', 'day', 'month', 'week', 'holiday']
	Types of features in processed data (raw d

[1000]	valid_set's l2: 0.00372338	valid_set's SMAPE: -1.75431


	-0.3327	 = Validation score   (-SMAPE)
	0.35s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge ...
	-0.2944	 = Validation score   (-SMAPE)
	1.01s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.2695	 = Validation score   (-SMAPE)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2.2s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20230827_130643\")
No path specified. Models will be saved in: "AutogluonModels\ag-20230827_130645\"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_130645\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   126.08 GB / 498.62 GB (25.3%)
Train Data Rows:    1497
Train

[1000]	valid_set's l2: 0.00817329	valid_set's SMAPE: -1.49427


...

[1000]	valid_set's l2: 0.00766752	valid_set's SMAPE: -1.9884


	-0.5121	 = Validation score   (-SMAPE)
	0.38s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge ...
	-0.5156	 = Validation score   (-SMAPE)
	0.74s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.4561	 = Validation score   (-SMAPE)
	0.09s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2.0s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20230827_132206\")
No path specified. Models will be saved in: "AutogluonModels\ag-20230827_132208\"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_132208\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   124.39 GB / 498.62 GB (24.9%)
Train Data Rows:    1498
Train

[1000]	valid_set's l2: 0.00730574	valid_set's SMAPE: -1.85219


	-0.4502	 = Validation score   (-SMAPE)
	0.69s	 = Training   runtime
	0.01s	 = Validation runtime
Fitting model: LightGBM ...
	-0.4897	 = Validation score   (-SMAPE)
	0.56s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: LightGBMLarge ...
	-0.5441	 = Validation score   (-SMAPE)
	0.76s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.4494	 = Validation score   (-SMAPE)
	0.08s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 2.29s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20230827_132208\")
No path specified. Models will be saved in: "AutogluonModels\ag-20230827_132210\"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_132210\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform M

2.282570323855048

In [ ]:
#train_x_type["건물번호"] = train_x_type["건물번호"].astype("category")
#test_x_type["건물번호"] = test_x_type["건물번호"].astype("category")

valid_df_type = train_x_type.copy()
valid_df_type['전력소비량(kWh)'] = 0
valid_df_type['pred'] = 0
pred_df_type = test_x_type.copy()
pred_df_type['전력소비량(kWh)'] = 0

labels_type = []

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

for i in tqdm(train_x_type['건물유형'].unique()):
    X = train_x_type[train_x_type['건물유형'] == i].drop(columns='건물유형')
    y = train_y[train_x_type['건물유형'] == i]
    valid_df_type.loc[train_x_type['건물유형'] == i, '전력소비량(kWh)'] = y
    y = np.log1p(y)
    x_test = test_x_type[test_x_type['건물유형'] == i].drop(columns='건물유형')

    tmp_preds = []
    for alpha in alphas:

        results = []

        for fold_, (train_index, val_index) in enumerate(skf.split(X, X['day'])):

            x_train, x_valid = X.iloc[train_index], X.iloc[val_index]
            y_train, y_valid = y.iloc[train_index], y.iloc[val_index]
            tr = pd.concat([x_train,y_train],axis=1)
            vl = pd.concat([x_valid,y_valid],axis=1)

            train_data = TabularDataset(tr)
            test_data = TabularDataset(x_test)

            predictor = TabularPredictor(label='전력소비량(kWh)',  eval_metric=mape_custom).fit(train_data,  tuning_data=vl, presets='medium_quality',  ag_args_fit={'num_gpus': 0}, excluded_model_types = ['CAT', 'NN_TORCH' ,'XGB' ,'RF', 'FASTAI', 'KNN', 'XT'])
            pred = predictor.predict(test_data, model='LightGBMLarge')


            val_pred = predictor.predict(x_valid)
            valid_df_type.loc[x_valid.index, 'pred'] += np.expm1(val_pred)
            pred = predictor.predict(x_test)
            results.append(pred)


        preds = np.mean(results, axis=0)
        preds = np.expm1(preds)
        tmp_preds.append(preds)

    preds = np.mean(tmp_preds, axis=0)
    pred_df_type.loc[test_x_type['건물유형'] == i, '전력소비량(kWh)'] = preds

    labels_type.append([valid_df_type.loc[train_x_type['건물유형'] == i, '전력소비량(kWh)'], preds])

# 3.462504576825228
valid_df_type['pred'] = valid_df_type['pred'] / len(alphas)
type_val_score = SMAPE(valid_df_type['전력소비량(kWh)'], valid_df_type['pred'])
type_val_score

  0%|          | 0/12 [00:00<?, ?it/s]No path specified. Models will be saved in: "AutogluonModels\ag-20230827_132231\"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_132231\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   124.34 GB / 498.62 GB (24.9%)
Train Data Rows:    22464
Train Data Columns: 25
Tuning Data Rows:    5616
Tuning Data Columns: 25
Label Column: 전력소비량(kWh)
Preprocessing data ...
AutoGluon infers your prediction problem is: 'regression' (because dtype of label-column == float and many unique label-values observed).
	Label info (max, min, mean, stddev): (8.948409213704661, 5.700711096131715, 7.47065, 0.51805)
	If 'regression' is not the correct problem_type, please manually specify the problem_type parameter during predictor init (You may specify problem_type as one of: ['binary',

[1000]	valid_set's l2: 0.00670837	valid_set's SMAPE: -1.76724
[2000]	valid_set's l2: 0.00578718	valid_set's SMAPE: -1.76004
[3000]	valid_set's l2: 0.0053492	valid_set's SMAPE: -1.75439
[4000]	valid_set's l2: 0.00509966	valid_set's SMAPE: -1.7528


	-0.3157	 = Validation score   (-SMAPE)
	7.0s	 = Training   runtime
	0.23s	 = Validation runtime
Fitting model: LightGBM ...
	-0.3474	 = Validation score   (-SMAPE)
	1.54s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: LightGBMLarge ...


[1000]	valid_set's l2: 0.00499095	valid_set's SMAPE: -1.75786


	-0.2958	 = Validation score   (-SMAPE)
	5.01s	 = Training   runtime
	0.1s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	-0.2946	 = Validation score   (-SMAPE)
	0.11s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 14.62s ... Best model: "WeightedEnsemble_L2"
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels\ag-20230827_132231\")
No path specified. Models will be saved in: "AutogluonModels\ag-20230827_132246\"
Presets specified: ['medium_quality']
Beginning AutoGluon training ...
AutoGluon will save models to "AutogluonModels\ag-20230827_132246\"
AutoGluon Version:  0.8.2
Python Version:     3.8.6
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19041
Disk Space Avail:   124.30 GB / 498.62 GB (24.9%)
Train Data Rows:    22464
Train Data Columns: 25
Tuning Data Rows:    5616
Tuning Data Columns: 25
Label Column: 전력소비량(kWh)
Preprocessing data ...
AutoGluon i

...

[1000]	valid_set's l2: 0.00628949	valid_set's SMAPE: -1.76469
[2000]	valid_set's l2: 0.00542288	valid_set's SMAPE: -1.75738
[3000]	valid_set's l2: 0.00503688	valid_set's SMAPE: -1.75611
[4000]	valid_set's l2: 0.00477849	valid_set's SMAPE: -1.75359
[5000]	valid_set's l2: 0.00460862	valid_set's SMAPE: -1.75225
[6000]	valid_set's l2: 0.00449098	valid_set's SMAPE: -1.75111
[7000]	valid_set's l2: 0.00440404	valid_set's SMAPE: -1.75121


	-0.2953	 = Validation score   (-SMAPE)
	10.4s	 = Training   runtime
	0.34s	 = Validation runtime
Fitting model: LightGBM ...


[1000]	valid_set's l2: 0.00533206	valid_set's SMAPE: -1.76393
[2000]	valid_set's l2: 0.00477932	valid_set's SMAPE: -1.76088


	-0.308	 = Validation score   (-SMAPE)
	3.18s	 = Training   runtime
	0.09s	 = Validation runtime
Fitting model: LightGBMLarge ...
  0%|          | 0/12 [01:08<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# (1.9531097965617121, 2.557635430408023, 2.2553726134848677)
# (1.9530552780909811, 2.5519303451985462, 2.2524928116447636)
# (1.7183125894172449, 2.287520125412716, 2.00291635741498)
val_score, type_val_score, np.mean([val_score, type_val_score])

(2.2340621785608286, 2.748818609952282, 2.4914403942565553)

In [ ]:
from sklearn.linear_model import LinearRegression

counts = 0
w = 0.97
# 1.9803304293430894
for num in train_df['건물번호'].unique():
    X = pd.concat([valid_df.loc[valid_df['건물번호'] == num, 'pred'], valid_df_type.loc[valid_df_type['건물번호'] == num, 'pred']], axis=1).values
    y = valid_df.loc[valid_df['건물번호'] == num, '전력소비량(kWh)']
    reg = LinearRegression().fit(X, y)
    p = reg.predict(X)
    score = SMAPE(valid_df.loc[valid_df['건물번호'] == num, '전력소비량(kWh)'], p)
    # print(f'{num}, score: {score}')
    pred_x = pd.concat([pred_df.loc[pred_df['건물번호'] == num, '전력소비량(kWh)'], pred_df_type.loc[pred_df['건물번호'] == num, '전력소비량(kWh)']], axis=1).values
    pred = reg.predict(pred_x)

    threshold = pd.pivot_table(train_df.loc[train_df['건물번호'] == num], values = '전력소비량(kWh)', index = ['day', 'hour'], aggfunc = min).reset_index()
    df_pred = pred_df.loc[pred_df['건물번호'] == num].reset_index(drop=True)
    for j in range(len(pred)):
        min_power = threshold.loc[(threshold.day == df_pred.day[j])&(threshold.hour == df_pred.hour[j]), '전력소비량(kWh)'].values[0]
        if pred[j] < min_power:
            pred[j] = min_power
            counts += 1

    threshold = pd.pivot_table(train_df.loc[train_df['건물번호'] == num], values = '전력소비량(kWh)', index = ['day', 'hour'], aggfunc = max).reset_index()
    df_pred = pred_df.loc[pred_df['건물번호'] == num].reset_index(drop=True)
    for j in range(len(pred)):
        max_power = threshold.loc[(threshold.day == df_pred.day[j])&(threshold.hour == df_pred.hour[j]), '전력소비량(kWh)'].values[0]
        if pred[j] > max_power:
            pred[j] = max_power
            counts += 1

    pred_df.loc[pred_df['건물번호'] == num, '전력소비량(kWh)'] = pred

    valid_df.loc[valid_df['건물번호'] == num, 'pred'] = p

SMAPE(valid_df['전력소비량(kWh)'], valid_df['pred'])

In [ ]:
counts # 573 538

130

In [ ]:
type_val_score / (val_score + type_val_score), SMAPE(valid_df['전력소비량(kWh)'], valid_df['pred'])

(0.5516524931298885, 1.5506366284519835)

In [ ]:
type_val_score / (val_score + type_val_score), SMAPE(valid_df['전력소비량(kWh)'], valid_df['pred'])

(0.5530377787563981, 1.5445747552176554)

In [ ]:
# plt.figure(figsize=(26, 300))
# for i in train_df['건물번호'].unique():
#     plt.subplot(train_df['건물번호'].nunique(), 1, i)
#     plt.title(i)
#     plt.plot(range(len(labels[i-1][0])), labels[i-1][0])
#     plt.plot(range(len(labels[i-1][0]), len(labels[i-1][0])+len(labels[i-1][1])), pred_df[pred_df['건물번호']==i]['전력소비량(kWh)'])
# plt.show()

In [ ]:
submission = pd.read_csv('sample_submission.csv')
submission['answer'] = pred_df['전력소비량(kWh)']
submission.to_csv('jh_21011928_AG.csv', index=False)
submission